In [4]:
from tqdm import tqdm
import numpy as np
import torch
from torch.utils.data import DataLoader
import config

from datasets.aileen_object import AileenObjectDataset
from model import AttributeNetwork, VGG_Net, RES_Net

In [5]:
shapes = ['cone', 'box', 'cylinder', 'sphere', 'capsule']
sizes = ['small', 'medium', 'large']
colors = ['blue', 'red', 'green', 'yellow', 'purple']

testData = AileenObjectDataset(config.ROOT_TEST)
test_size = len(testData)
test_loader = DataLoader(dataset=testData, shuffle=False, batch_size=1)

In [6]:
model = VGG_Net(13)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    model.cuda()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/mfan/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [7]:
print('Load weights...')
model.load_state_dict(torch.load('attribute_best_model.pth'))
print('Weights are loaded...')
model.to(device)

Load weights...
Weights are loaded...


VGG_Net(
  (main): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16): M

In [10]:
shape_correct = 0
size_correct = 0
color_correct = 0
with torch.no_grad():
    for x_batch, y_batch in tqdm(test_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_test_pred = model(x_batch)
        # _, y_pred_tag = torch.max(y_test_pred, dim = 1)
        y_test_pred = y_test_pred.cpu().numpy().copy()
        pred_shape = y_test_pred[0][0:5]
        pred_size = y_test_pred[0][5:8]
        pred_color = y_test_pred[0][8:13]
        flag = np.bool(y_test_pred[0][1])
        index_max = np.where(pred_shape == np.max(pred_shape))[0][0]
        # print pred_shape
        shape = shapes[np.where(pred_shape == np.max(pred_shape))[0][0]]
        size = sizes[np.where(pred_size == max(pred_size))[0][0]]
        color = colors[np.where(pred_color == max(pred_color))[0][0]]

        gt_y = y_batch.cpu().numpy().copy()
        gt_shape = gt_y[0][0:5]
        gt_size = gt_y[0][5:8]
        gt_color = gt_y[0][8:13]
        # print gt_y
        # print np.where(gt_shape == np.max(gt_shape))
        gtshape = shapes[np.where(gt_shape == np.max(gt_shape))[0][0]]
        gtsize = sizes[np.where(gt_size == np.max(gt_size))[0][0]]
        gtcolor = colors[np.where(gt_color == np.max(gt_color))[0][0]]
        if shape == gtshape:
            shape_correct += 1
        if size == gtsize:
            size_correct += 1
        if color == gtcolor:
            color_correct += 1
        print("****************")
        print("predicted shape: %s    gt shape: %s" % (shape, gtshape))
        print("predicted size: %s    gt size: %s" % (size, gtsize))
        print("predicted color: %s    gt color: %s" % (color, gtcolor))
        print("has round surface: %r" % flag)
        print("****************")

  0%|                                                  | 0/7676 [00:00<?, ?it/s]/tmp/ipykernel_12081/238456162.py:13: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  flag = np.bool(y_test_pred[0][1])
  0%|▏                                       | 28/7676 [00:00<00:56, 135.36it/s]

****************
predicted shape: sphere    gt shape: sphere
predicted size: medium    gt size: medium
predicted color: blue    gt color: blue
has round surface: True
****************
****************
predicted shape: sphere    gt shape: sphere
predicted size: medium    gt size: medium
predicted color: purple    gt color: purple
has round surface: True
****************
****************
predicted shape: box    gt shape: box
predicted size: medium    gt size: medium
predicted color: green    gt color: green
has round surface: True
****************
****************
predicted shape: cone    gt shape: cone
predicted size: medium    gt size: medium
predicted color: yellow    gt color: yellow
has round surface: True
****************
****************
predicted shape: box    gt shape: box
predicted size: large    gt size: large
predicted color: red    gt color: red
has round surface: True
****************
****************
predicted shape: sphere    gt shape: sphere
predicted size: large    gt s

  1%|▏                                       | 42/7676 [00:00<00:57, 132.65it/s]

****************
predicted shape: box    gt shape: box
predicted size: medium    gt size: medium
predicted color: blue    gt color: blue
has round surface: True
****************
****************
predicted shape: cone    gt shape: cone
predicted size: large    gt size: large
predicted color: yellow    gt color: yellow
has round surface: True
****************
****************
predicted shape: box    gt shape: box
predicted size: large    gt size: large
predicted color: green    gt color: green
has round surface: True
****************
****************
predicted shape: box    gt shape: box
predicted size: small    gt size: small
predicted color: blue    gt color: blue
has round surface: True
****************
****************
predicted shape: sphere    gt shape: sphere
predicted size: large    gt size: large
predicted color: red    gt color: red
has round surface: True
****************
****************
predicted shape: box    gt shape: box
predicted size: large    gt size: large
predicted c

  1%|▎                                       | 71/7676 [00:00<00:54, 139.01it/s]

...

****************
predicted shape: cone    gt shape: cone
predicted size: medium    gt size: medium
predicted color: yellow    gt color: yellow
has round surface: True
****************
****************
predicted shape: box    gt shape: box
predicted size: small    gt size: small
predicted color: green    gt color: green
has round surface: True
****************
****************
predicted shape: sphere    gt shape: sphere
predicted size: small    gt size: small
predicted color: blue    gt color: yellow
has round surface: True
****************
****************
predicted shape: capsule    gt shape: capsule
predicted size: large    gt size: large
predicted color: red    gt color: red
has round surface: True
****************
****************
predicted shape: sphere    gt shape: cone
predicted size: small    gt size: small
predicted color: yellow    gt color: purple
has round surface: True
****************
****************
predicted shape: capsule    gt shape: capsule
predicted size: small    

100%|██████████████████████████████████████▉| 7661/7676 [01:00<00:00, 99.22it/s]

****************
predicted shape: sphere    gt shape: sphere
predicted size: large    gt size: large
predicted color: yellow    gt color: yellow
has round surface: True
****************
****************
predicted shape: box    gt shape: box
predicted size: small    gt size: small
predicted color: blue    gt color: blue
has round surface: True
****************
****************
predicted shape: cone    gt shape: cone
predicted size: medium    gt size: medium
predicted color: yellow    gt color: yellow
has round surface: True
****************
****************
predicted shape: box    gt shape: box
predicted size: large    gt size: large
predicted color: green    gt color: green
has round surface: True
****************
****************
predicted shape: cone    gt shape: cone
predicted size: small    gt size: small
predicted color: green    gt color: green
has round surface: True
****************
****************
predicted shape: cylinder    gt shape: cylinder
predicted size: medium    gt si

100%|██████████████████████████████████████| 7676/7676 [01:00<00:00, 127.18it/s]

****************
predicted shape: cone    gt shape: cone
predicted size: small    gt size: small
predicted color: purple    gt color: purple
has round surface: True
****************
****************
predicted shape: sphere    gt shape: sphere
predicted size: large    gt size: large
predicted color: red    gt color: red
has round surface: True
****************
****************
predicted shape: box    gt shape: box
predicted size: large    gt size: large
predicted color: yellow    gt color: yellow
has round surface: True
****************


In [11]:
print("shape prediction accuracy = %f" % (shape_correct / test_size))
print("size prediction accuracy = %f" % (size_correct / test_size))
print("color prediction accuracy = %f" % (color_correct / test_size))

shape prediction accuracy = 0.948020
size prediction accuracy = 0.986712
color prediction accuracy = 0.950234
